# Introduction

Idea behind finding genuine reviews from data sets:

> Part 1: Prediction of Helpfulness from given data i.e if people found review helpful or not.

> Part 2: Classification of genuine and fake/sarcastic reviews.

Data Preparation
--

Data preprocessing for "Preparing data" for the Model.

In [1]:
# Importing the required files.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pandas as pd
import string
from time import time
import nltk
from nltk.corpus import stopwords 
stops = set(stopwords.words("english"))
import re
from IPython.display import display # Allows the use of display() for DataFrames
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
RAN_STATE = 42 # Setting the random state

Now, reading of the data.

The dataset is a JSON file -> use the-> read_json() function of Pandas.
Use *lines=True*  to read the file as a JSON object per line or else it will show an error.

Content of dataset -
* 5-core dataset of product reviews from Amazon Electronics category from May 1996 - July 2014. 
* Contains total of 1689188 entries. 
* Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.

Columns are:
* asin (Amazon Standard Identification Number)- ID of the product, like B000FA64PK (basically a aplhanumeric number for product identification)
* helpful - helpfulness rating of the review - example: 2/3.
* overall - rating of the product.
* reviewText - text of the review (heading).
* reviewTime - time of the review (raw).
* reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
* reviewerName - name of the reviewer.
* summary - summary of the review (description).
* unixReviewTime - unix timestamp.

In [2]:
# Read json data
data=pd.read_json('Review_Set5.json',lines=True,orient='columns')
data

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1,"Major issues, only excuses for support",1317254400,"09 29, 2011"
...,...,...,...,...,...,...,...,...,...
1689183,A34BZM6S9L7QI4,B00LGQ6HL8,"Candy Cane ""Is it just me?""","[1, 1]",Burned these in before listening to them for a...,5,Boom -- Pop -- Pow. These deliver.,1405555200,"07 17, 2014"
1689184,A1G650TTTHEAL5,B00LGQ6HL8,"Charles Spanky ""Zumina Reviews""","[0, 0]",Some people like DJ style headphones or earbud...,5,"Thin and light, without compromising on sound ...",1405382400,"07 15, 2014"
1689185,A25C2M3QF9G7OQ,B00LGQ6HL8,Comdet,"[0, 0]",I&#8217;m a big fan of the Brainwavz S1 (actua...,5,Same form factor and durability as the S1 with...,1405555200,"07 17, 2014"
1689186,A1E1LEVQ9VQNK,B00LGQ6HL8,J. Chambers,"[0, 0]","I've used theBrainwavz S1 In Ear Headphones, a...",5,Superb audio quality in a very comfortable set...,1405641600,"07 18, 2014"


using the drop_duplicate() function to remove duplicates of reviews.


In [3]:
# Remove duplicate reviews(if any)
print("The shape of the data set before removing duplicate reviews : {}".format(data.shape))
data=data.drop_duplicates(subset=["reviewText"], keep='first', inplace=False)
print("The shape of the data set after removing duplicate reviews : {}".format(data.shape))

The shape of the data set before removing duplicate reviews : (1689188, 9)
The shape of the data set after removing duplicate reviews : (1687169, 9)


duplicate reviews removed. 
Now, time for text preprocessing.

Text Preprocessing
--

We need to preprocess the data because the same preprocessed data will be needed for step 1 and 2

Preprocessing the whole data took approximately an hour.

In [4]:
def preprocess(x):
    x = x.replace(",000,000", " m").replace(",000", " k").replace("′", "'").replace("’", "'")\
                           .replace("won't", " will not").replace("cannot", " can not").replace("can't", " can not")\
                           .replace("n't", " not").replace("what's", " what is").replace("it's", " it is")\
                           .replace("'ve", " have").replace("'m", " am").replace("'re", " are")\
                           .replace("he's", " he is").replace("she's", " she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will").replace("how's"," how has").replace("y'all"," you all")\
                           .replace("o'clock"," of the clock").replace("ne'er"," never").replace("let's"," let us")\
                           .replace("finna"," fixing to").replace("gonna"," going to").replace("gimme"," give me").replace("gotta"," got to").replace("'d"," would")\
                           .replace("daresn't"," dare not").replace("dasn't"," dare not").replace("e'er"," ever").replace("everyone's"," everyone is")\
                           .replace("'cause'"," because").replace("i'm"," i am")
    
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    x=re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))',' ',x)
    x=re.sub(r"\\s*\\b(?=\\w*(\\w)\\1{2,})\\w*\\b",' ',x)
    x=re.sub(r'<.*?>',' ',x)
    x=re.sub('[^a-zA-Z]',' ',x)
    x=''.join([i for i in x if not i.isdigit()])
    return x

Now, 
1. convert reviews to lowercase. 
2. Then we are going to do preprocessing. 
3. And finally, go to lemmatization.

lemmatization over stemming is preferred because it extracts by considering vocabulary rather than just the root word in stemming.


In [ ]:
# Import libraries
from nltk.corpus import stopwords
from textblob import TextBlob
from textblob import Word

# Lower casing and removing punctuations
data['reviewText'] = data['reviewText'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data['reviewText'] = data['reviewText'].str.replace('[^\w\s]','')

# Removing stopwords
#stop = stopwords.words('english')
#data['reviewText'] = data['reviewText'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data['reviewText'] = data['reviewText'].apply(lambda x: preprocess(x))
# Lemmatization

data['reviewText'] = data['reviewText'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data.reviewText.head(5)

Now, let's start with Steps.

# Part 1

Idea behind this step
--
Amazon provides a system to rate the helpfulness of a review, but the problem with amazon is latest review is added at the top of the forum and despite of the rating it is not moved down or above.

The Group goal is to create a model using machine learning techniques that would pre-rate the helpfulness of a given customer review before they are posted on the top of the forum. 
This way poor quality reviews would not be shown on top of forums.

The model is trained on Amazon Reviews to predict if a given review is helpful or not helpful.

Reading the data to check what we have at hand.

In [ ]:
data.head()

**Targetted data columns:**
> Input Features : reviewText,overall

> Output labels: helpfulness

**Reason for selecting input features:** When we give any review, along with text(reviewText) of the review we also give a rating in stars(overall).

**the helpful column:** helpful column given above is a list containing two values---no of helpful ratings and the total no of ratings--- separated by a comma.

We are dividing the helpful column into two parts i.e. 
 
 > helpful_numerator => contains no. of helpful rating.
 
 > helpful_denominator => contains total no. of ratings.

and then we are deleting the helpful column.
Each column will have a single value data making it easier to operate on.

In [ ]:
#select the columns
df = data.iloc[:, [5,4,3]]

#split numerator and denominator
df['helpful_numerator'] = df['helpful'].apply(lambda x: x[0])
df['helpful_denominator'] = df['helpful'].apply(lambda x: x[1])

# delete un-needed helpful column
del df['helpful']

#Check if we have any null values
print (df.isnull().sum())

overall                0

reviewText             0

helpful_numerator      0

helpful_denominator    0

dtype: int64

just a quick stats check.

In [ ]:
df.describe()

Here, the dataset is huge(about 1687169 records), not all records are worth taking into consideration.
So to narrow it down, take only those records that have at least 20 ratings in total.
it is just based on the assumption that people can naturally spot the useless review and also our systems can't handle that much data for testing :(

In [ ]:
# include reviews that have more than 20 helpfulness data point only
df1 = df[(df.helpful_denominator > 20)].copy()
df1.shape

Now, to get our output label **helpfulness** which is the ratio of 

**helpfulness=helpful_numerator/helpful_denominator**.

The result is compared with a threshold value(*let's keep threshold as 50%, but we can change it later per our requirement*).
* If result > threshold ==> helpful = 1
* if result < treshold ==> not helpful = 0

In [ ]:
# transform Helpfulness into a binary variable with 0.5 ratio
threshold = 0.5s
df1.loc[:, 'Helpful'] = np.where(df1.loc[:, 'helpful_numerator'] \
                                 / df1.loc[:, 'helpful_denominator'] > threshold, 1, 0)
df1.head(3)

Now, count the data to get total the distribution of helpfulness.

In [ ]:
#Check the balance
print ('Count:')
display(df1.groupby('Helpful').count())

Final check on the data, since we are going to creating our model.

In [ ]:
df1

We are ready with the dataset we need to operate on.
Now, directly applying TF-IDF Vectorizer to generate more features.
TF-IDF is an acronym of Term Frequency Inverse Document Frequency. It is a statistical measure used to find how important a word is to document in a collection or corpus. It is generally used in text mining and information retrieval.
to operate on our review text data

> TF: Term Frequency, which measures how frequently a term occurs in a document. 
>  
> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
>  
> IDF: Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important.:
>  
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# define the vectorizer
vectorizer = TfidfVectorizer(min_df = 0.01)
# fit the vectorizers to the data.
features = vectorizer.fit_transform(df1['reviewText'])
features

Since we don't have a separate dataset for testing we are splitting data as 80% for training and 20% for testing.

In [ ]:
# split and shuffle data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features,df1['Helpful'], test_size=0.2, random_state=RAN_STATE)

Since our problem is of binary classification(helpful or not helpful), we are using roc_auc_score to evaluate our model.

The roc_auc_score computes the area under the receiver operating characteristic (ROC) curve which is also denoted by AUC or AUROC. By computing the area under the roc curve, the curve information is summarized in one number.

This curve is created by plotting the true positive rate (TPR) against the false positive rate (FPR). The area under the curve is used to give a score to the model.
> If AUC = 0.5 => TPR = FPR, and the model is doing just random computations.
>
> If AUC= 1.0 => TPR = 100%,and it is an ideal model.


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on roc_auc score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    probas = clf.predict_proba(features)
    end = time()
    
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    return roc_auc_score(target.values, probas[:,1].T)


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on roc_auc score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, X_train.shape[0]))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print ("ROC_AUC score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("ROC_AUC score for test set: {:.4f}.\n".format(predict_labels(clf, X_test, y_test)))
    
def clf_test_roc_score(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    probas = probas =clf.predict_proba(X_test)
    return roc_auc_score(y_test, probas[:,1].T)

To make a baseline model for our project we are going to use the following algorithms :

* **GaussianNB**: 
<img src="https://imgur.com/nMRnwlL.png" width="400px">

* **Logistic Regression**:
* **Decision tree**: 

All the algorithms used here are popular algorithms for classification. However, since we are doing binary classification,
Let's see which out of 3 gives better result: *Logistic Regression*, *Decision tree* or *Gaussian Naive Bayes*.


In [ ]:
# Import the supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


# Initialize the models using a random state were applicable.
clf_list = [GaussianNB(),  
            LogisticRegression(random_state = RAN_STATE),
            DecisionTreeClassifier(random_state = RAN_STATE)]
x_tr = X_train.toarray()
x_te = X_test.toarray()


# Set up the training set sizes for 10000, 20000 and 40000 respectively.
train_feature_list = [x_tr[0:10000],x_tr[0:20000],x_tr]
train_target_list = [y_train[0:10000], y_train[0:20000], y_train]


# Execute the 'train_predict' function for each of the classifiers and each training set size
for clf in clf_list:
    for a, b in zip(train_feature_list, train_target_list):
        train_predict(clf, a, b, x_te, y_test)

It's a hassle to find each value. So let's just visualize the outputs for all models. Graphs make it's easier to understand. 

In [ ]:
FIG_SIZE = (14,8)
# Visualize all of the classifiers                                                               
for clf in clf_list:
    x_graph = []
    y_graph = []
    for a, b in zip(train_feature_list, train_target_list):
        y_graph.append(clf_test_roc_score(clf, a, b, x_te, y_test))
        x_graph.append(len(a))
    plt.scatter(x_graph,y_graph)
    plt.plot(x_graph,y_graph, label = clf.__class__.__name__)

plt.title('Comparison of Different Classifiers')
plt.xlabel('Training Size')
plt.ylabel('ROC_AUC score on test set')
plt.legend(bbox_to_anchor=(1.6, 1.05))
plt.figure(figsize=FIG_SIZE)             
plt.show() 

Logistic Regression gives us the best accuracy. 
>Its final score for the area under the ROC curve was 0.8704 and a sample size of ~40,000.

>Besides, it is the fastest. The training speed and prediction speed were 19.993s and 0.955s  for a sample size of 40,000. 

>Since our model has to consider the accuracy and speed, the Logistic Regression algorithm represents the ideal model for us.

Now, let's add values of scores to the review text and see if we can increase the accuracy of our model.

In [ ]:
#add Score column to features
import scipy as scipy

overall = np.array(list(df1.overall))
overall = overall.reshape(features.shape[0], 1)

features = scipy.sparse.hstack((features,scipy.sparse.csr_matrix(overall)))

features = scipy.sparse.csr_matrix(features)
features


We can now split the dataset and try to optimize our initial model.

In [ ]:
X_train2, X_test2, y_train, y_test = train_test_split(features, df1['Helpful'], test_size=0.2, random_state=RAN_STATE)

We have used hyper-parameter, Hyper-parameters are parameters that are not directly learned within estimators. In scikit-learn, they are passed as arguments to the constructor of the estimator classes. Hyperparameter tuning helps us in optimizing our model.


We will now be applying Gridsearch and Cross-Validation techniques to optimize and hypertune our model.

* **GridSearch:** Exhaustive search over specified parameter values for an estimator.

* **Cross-Validation:** In the train-test split, we use only 20% for testing. The performance metric we get on that 20% test data may not be accurate. So Cross-Validation allows you to consume 100% of the data for training and testing both.


In [ ]:
from sklearn.model_selection import GridSearchCV,cross_validate,StratifiedKFold
#make the grid search object
gs2 = GridSearchCV(
    estimator=LogisticRegression(),
    param_grid={'C': [10**i for i in range(-5,5)], 'class_weight': [None, 'balanced']},
    cv=StratifiedKFold(n_splits=5),
    scoring='roc_auc'
)

#fit the grid search object to our new dataset
print ('Fitting grid search...')
gs2.fit(X_train2, y_train)
print ("Grid search fitted.")

Let's the best estimator for our model.

In [ ]:
#print the grid search scores.
gs2.best_params_

We can see our optimized classifier is a LogisticRession with a 'C' parameter of 1 and a 'class_weight' = 'None'. This is the same as default, meaning our optimization step did not change the parameters of our model. Let's now find our ROC_AUC Score.

In [ ]:
clf2 = gs2.best_estimator_
probas =clf2.predict_proba(X_test2)


# ROC/AUC score
print ('ROC_AUC Score:',roc_auc_score(y_test, probas[:,1].T))

Wow! 90 % ROC_AUC Score. That means our model has been trained well.

Let's now plot the graph to find the ROC Curve for Helpful Rating of both TFIDF and TFIDF along with the overall score. 

In [ ]:

clf = LogisticRegression()
clf.fit(X_train,y_train)
probas = clf.predict_proba(X_test)
clf2 = gs2.best_estimator_
probas2 =clf2.predict_proba(X_test2)
plt.figure(figsize = FIG_SIZE)

# plot graph to show roc_auc_score.
plt.plot(roc_curve(y_test, probas[:,1])[0], roc_curve(y_test, probas[:,1])[1], label = 'TFIDF')
plt.plot(roc_curve(y_test, probas2[:,1])[0], roc_curve(y_test, probas2[:,1])[1], label = 'TFIDF + overall')
plt.title('ROC Curve for Helpful Rating')
plt.grid()
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')

plt.legend(bbox_to_anchor=(1.0, .5))
plt.figure(figsize=FIG_SIZE)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
clf = LogisticRegression(random_state=0)
clf.fit(X_train, y_train)
LogisticRegression(random_state=0)
plot_confusion_matrix(clf, X_test, y_test)  
plt.show()

X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)
DecisionTreeClassifier(random_state=0)
plot_confusion_matrix(clf, X_test, y_test)  
plt.show()

X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
clf = GaussianNB()
clf.fit(X_train, y_train)
plot_confusion_matrix(clf, X_test, y_test)  
plt.show()

In [ ]:
X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
clf = LogisticRegression(random_state=0)
clf.fit(X_train, y_train)
LogisticRegression(random_state=0)
LogisticRegression = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = LogisticRegression.predict(X_test)


#importing confusion matrix
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test, y_pred, target_names=['Class 1', 'Class 2']))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)
DecisionTreeClassifier(random_state=0)
DecisionTreeClassifier = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
y_pred = DecisionTreeClassifier.predict(X_test)


#importing confusion matrix
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test, y_pred, target_names=['Class 1', 'Class 2']))

In [ ]:
X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
clf = GaussianNB()
clf.fit(X_train, y_train)
GaussianNB = GaussianNB().fit(X_train, y_train)
y_pred = GaussianNB.predict(X_test)


#importing confusion matrix
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')

As we can see in the above ROC Curve, adding an overall score to TFIDF vectors would give us a higher Area Under Curve(AUC).

Conclusion:
--
The important quality of this step is the 'effect of introducing a new key feature to our model'.
Here, we mainly looked at the TFIDF features generated from Amazon's Electronic Reviews text and added the 'overall_rating' (star rating) that was given to the product by the reviewer. 
We used these features to predict how 'helpful' other users would find the review.

# Part 2

Thought Process behind this Step
--
The problem that is being addressed here is if Amazon Review is fake or not genuine. Sometimes some people give fake reviews on forums. This could harm the sale of the product if such a review came on top. 

**how we see fake review?**
fake review: means a review which is given a 5-star rating but in review text, the person gives a negative review and since it is rated 5 stars, so chances are that review would come on top of the forum. 

Possibility exists that such reviews be given by competitors to harm the sale of the product.
**Our solution to this:**
Create a model using machine learning techniques that would classify given customer reviews based on sentiment analysis before they are posted on the top of the forum. This way fake reviews would not be shown on top of forums.

The model will be trained on Amazon Reviews to classify if a given review is genuine or not.

Now, let's run a quick data check.

In [ ]:
data.head()

Here, we are going to select **reviewText** and **overall** score rating since we going to classify reviews based on these two features.

In [ ]:
#select the columns
df2 = data.iloc[:, [5,4]]
df2.head(10)

How we classified the ratings as Positive, Negative, and Neutral.

> * If the **score rating => 4 or 5**,we are taking it as **Positive**.
>
> * If the **score rating => 3**,we are taking it as **Neutral**.
>
> * And if the **score rating => 1 or 2**,we are taking it as **Negative**.

Then we are saving the result in a column named **Overall_Sentiment**. This would tell us the sentiment of the review based on the star rating.

In [ ]:
def score_classify(x):
    if x>3:
        return 'Positive'
    elif x<3:
        return 'Negative'
    else:
        return 'Neutral'
df2['Overall_Sentiment']=df2.apply(lambda x: score_classify(x['overall']),axis=1)
df2

Now, let's find out how the number of Positive, Negative, and Neutral reviews.

In [ ]:
df2.Overall_Sentiment.value_counts()

Now, we can see that 1354351 reviews are Positive,190693 are Negative while 142125 are Neutral.

But it may also be possible that some data may be missing or null. 

So we will now drop 'null values' if there are any. Then recheck the number of reviews.

In [ ]:
df2.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)
df2.Overall_Sentiment.value_counts()

We can see that there are no missing or null values.


Now,it's time for sentiment analysis of review text.

We will use Sentiment analysis to computationally analyze and identify opinions and judgments of a customer from a piece of text.
To understand if a piece of text is positive, negative, or neutral, based on their sentiment analysis.

We are using aspect-based sentiment analysis here.

Aspect-based sentiment analysis is generally for one or more aspects of a service or product. 

To a depth on how the customers feel about specific attributes of the product. 



Aspect-based sentiment analysis can be implemented in different ways. 

We tried Sentiment Intensity Analyzer(commonly known as SIA) from vaderSentiment. 

But the found that sentiment from Textblob gives better results than SIA here. So we are stick to the sentiment from Textblob.



In [ ]:
from textblob import TextBlob

sentiment_score_list = []
sentiment_label_list = []

for i in df2['reviewText'].values.tolist():
    sentiment_text=TextBlob(i)
    sentiment_score = sentiment_text.sentiment.polarity
    #print(sentiment_score)

    if sentiment_score > 0:
        sentiment_score_list.append(sentiment_score)
        sentiment_label_list.append('Positive')
    elif sentiment_score < 0:
        sentiment_score_list.append(sentiment_score)
        sentiment_label_list.append('Negative')
    else:
        sentiment_score_list.append(sentiment_score)
        sentiment_label_list.append('Neutral')
    
df2['Review_Sentiment'] = sentiment_label_list
df2['sentiment score'] = sentiment_score_list

display(df2.head(10))

We can see that some reviews are of Neutral, sentiment based on star rating bit are Positive, based on the sentiment of review text.

Neutral reviews could be classified wrongly.

So we are removing all Neutral reviews.

In [ ]:
before=df2.shape[0]
df2 = df2[df2.Overall_Sentiment != 'Neutral']
df2 = df2[df2.Review_Sentiment != 'Neutral']
df2.head(10)
after=df2.shape[0]
#EDA for finding how many neutral labels have been removed

Finding out how many Neutral reviews we have removed.

In [ ]:
print("The number of neutral labels have been removed : {}".format(before-after))

Significant amount of reviews have been removed.

Now, let's classify reviews as true or false.

**True review:** review in which the sentiment of star rating matches the sentiment of review text.

**False review:** review in which the sentiment of star rating does not matches the sentiment of review text..
Comparison done below.

In [ ]:
comparison_column = np.where(df2["Overall_Sentiment"] == df2["Review_Sentiment"], True, False)
df2["result"] = comparison_column
df2.head()

Finding the total number of all the fake reviews. 

And running the data check on the fake review data-set we generated.

In [ ]:
df2 = df2[df2.result != True]
print(df2.shape)
df2.head()

We found all the reviews that are not genuine and putting it figures it amounts to 202988 reviews.

>That is a lot of reviews.


>This part of the project is a SUCCESS.

Successfully implemented:
--
> we can easily find out if the given review is genuine or not.

>We used review text and star rating from Amazon's Reviews that was given to the product by the reviewer. 

>We used these features to classify if the given review is truly genuine or not.

